In [ ]:
import pandas as pd
import xarray as xr
from metpy.calc import geopotential_to_height
from metpy.units import units

import config
from utils import processing

In [ ]:
df = pd.read_csv(config.RAW_STORM_DB_PATH)
df.head()

In [ ]:
df = processing.rename_columns(df, column_map=config.COL_RENAME_MAP)

In [ ]:
df = processing.get_era5_indices_for_coords(
    df, era5_lons=config.ERA5_LONS, era5_lats=config.ERA5_LATS
)

In [ ]:
# load geopotential data and convert to height
geop = xr.open_dataset(config.DATA_DIR / "std" / "geop.nc")
geop_vals = geop["geop"].values.squeeze()
# source: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.geopotential_to_height.html#metpy.calc.geopotential_to_height
geop_units = units.Quantity(geop_vals, "m^2/s^2")
height = geopotential_to_height(geop_units)

# load subgrid oro angle data
anor = xr.open_dataset(config.DATA_DIR / "std" / "anor.nc")

df = processing.get_orography_features(df, height, anor)

In [ ]:
# select only the columns that are in the config
df = df[[col for col in config.FEATURE_COL_NAMES if col in df.columns]]

In [ ]:
df.to_csv(config.PROCESSED_DATASET_PATH, index=False)